In [272]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
import statsmodels.formula.api as smf

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots

In [273]:
pd.options.display.max_columns = None  # Remove "dots" from display when printing dataframes

# Read data

In [274]:
dfVehicles = pd.read_csv('vehicles.csv')

dfCounties = pd.read_csv('counties.csv', sep=';')
dfCrashesPoverty = pd.read_csv('crashes_poverty.csv', sep=';')

In [275]:
dfCounties.head(3)

,State,Abbreviation,Postal\ncode
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,American Samoa,,AS


In [276]:
dfCrashesPoverty.head(3)

,State,Number of Crashes,Poverty
0,Florida,1011,13.7
1,Tennessee,437,15.2
2,New Mexico,344,18.8


In [277]:
dfVehicles.sample(n=3).head(3)

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,description,county,state,lat,long
10560,des moines,8100,2008.0,nissan,pathfinder le 4x4,excellent,6 cylinders,gas,114000.0,clean,automatic,5N1BR18B78C611627,4wd,mid-size,SUV,brown,"Tan 2008 Nissan Pathfinder LE 4X4 with 113,000...",NaN,ia,41.6862,-93.9707
9926,memphis,18798,2017.0,nissan,rogue,NaN,4 cylinders,gas,15104.0,clean,automatic,NaN,fwd,compact,SUV,white,Use this link to see more information on this ...,NaN,tn,35.0259,-89.7886
10546,redding,17999,2008.0,mercedes-benz,sl-class,excellent,8 cylinders,gas,66000.0,clean,automatic,WDBSK71F88F136818,rwd,NaN,convertible,silver,www.ElegantAutoSales.netCall ☏ 916−794−3000SE ...,NaN,ca,38.6150,-121.2720


In [278]:
dfVehicles.dtypes

region           object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
vin              object
drive            object
size             object
type             object
paint_color      object
description      object
county          float64
state            object
lat             float64
long            float64
dtype: object

# Clear data

In [279]:
print(dfVehicles.shape)
dfVehicles = dfVehicles[dfVehicles['price'] >= 1]  # Delete free cars
dfVehicles = dfVehicles[dfVehicles['odometer'] <= 9999989]  # Delete top outlier
print(dfVehicles.shape)

(26988, 21)
(20226, 21)


In [280]:
print(dfVehicles.shape)

columns_fillnan = [
    'condition']  # 'manufacturer', 'condition', 'cylinders', 'fuel', 'transmission', 'type'

for i in columns_fillnan:
    dfVehicles = dfVehicles[dfVehicles[i].notna()]

print(dfVehicles.shape)

(20226, 21)
(12715, 21)


In [281]:
columns_drop = [
    'region', 'year', 'model',
    'title_status', #'vin',
    'paint_color', 'description', 'county',
    'lat', 'long']

for i in columns_drop:
    dfVehicles.drop([i], axis = 1, inplace=True)

In [282]:
# Replace null values
dfVehicles['transmission'] = dfVehicles['transmission'].fillna('NA')

In [283]:
dfVehicles.reset_index(inplace=True, drop=False)

## Merge all DataFrames

In [284]:
# Join dfCounties and dfCrashesPoverty
dfCounties = dfCounties.rename({
    'Postal\ncode':'postal_code'
    }, axis=1)

dfNew = pd.merge(dfCounties[['postal_code', 'State']], dfCrashesPoverty, how='outer', on='State')

# Replace null values
dfNew['Number of Crashes'] = dfNew['Number of Crashes'].fillna(0.0)
dfNew['Number of Crashes'] = dfNew['Number of Crashes'].astype(int)

del dfCounties
del dfCrashesPoverty

dfNew.sample(n=5).head()

,postal_code,State,Number of Crashes,Poverty
52,VT,Vermont,46,10.7
47,SC,South Carolina,244,15.2
14,ID,Idaho,142,11.7
42,OR,Oregon,125,12.5
32,NV,Nevada,153,13.1


In [285]:
# Join dfVehicles and dfNew
dfVehicles['state'] = dfVehicles['state'].apply(lambda x: x.upper())

dfVehicles = pd.merge(dfVehicles, dfNew, how='outer', left_on='state', right_on='postal_code')

dfVehicles.sample(n=5).head(5)

,index,price,manufacturer,condition,cylinders,fuel,odometer,transmission,vin,drive,size,type,state,postal_code,State,Number of Crashes,Poverty
5974,8999.0,3200.0,lexus,good,6 cylinders,gas,243000.0,automatic,2T2GA31U84C005646,fwd,mid-size,SUV,NC,NC,North Carolina,201,14.1
415,3576.0,18750.0,ford,good,6 cylinders,gas,40265.0,automatic,5R07A174029,NaN,NaN,coupe,CA,CA,California,116,12.8
7053,10807.0,6800.0,lexus,excellent,6 cylinders,gas,142000.0,automatic,NaN,fwd,full-size,sedan,TX,TX,Texas,268,14.9
12354,26823.0,10995.0,subaru,good,NaN,gas,143215.0,automatic,JF2GPAGCXD2845045,NaN,NaN,hatchback,AR,AR,Arkansas,143,16.8
7149,13412.0,8500.0,chevrolet,good,8 cylinders,gas,121000.0,automatic,NaN,4wd,full-size,truck,TX,TX,Texas,268,14.9


# EDA

In [286]:
dfVehicles['condition'].value_counts()

excellent    5997
good         5191
like new     1151
fair          312
new            41
salvage        23
Name: condition, dtype: int64

In [287]:
fig = px.histogram(dfVehicles, x='condition')
fig.show()

In [288]:
dfVehicles['odometer'].value_counts()

120000.0    53
150000.0    45
170000.0    44
0.0         44
125000.0    43
            ..
137524.0     1
22932.0      1
62372.0      1
137334.0     1
46.0         1
Name: odometer, Length: 8204, dtype: int64

In [289]:
fig = px.histogram(dfVehicles, x='odometer')  # , color='transmission'
fig.show()

In [290]:
dfVehicles['vin'].value_counts()

WDZPE7CC9D5767661    13
WP0CA2991XS652065     7
000000000000          5
2C3CDZBG9FH740753     5
1FA6P8TH4H5217450     5
                     ..
1FDXF47R88EE46271     1
1J4GR48K86C277840     1
3C4PDDEG5GT140672     1
2GKFLWEK5E6283688     1
JTLZE4FE8FJ078410     1
Name: vin, Length: 7367, dtype: int64

In [291]:
fig = px.scatter_matrix(dfVehicles, dimensions=[
    'condition','odometer', 'transmission', 'Number of Crashes', 'Poverty'])  #  , color='price'
fig.show()

## Completeness of the variables

Evalúe la completitud/exhaustividad de la variable y destaque los hallazgos que considere relevantes para una discusión.

In [292]:
odometer = len(dfVehicles['odometer'].unique().tolist())
condition = len(dfVehicles['condition'].unique().tolist())
vin = len(dfVehicles['vin'].unique().tolist())

print(f'Completitud de variables: \n \t Odómetro: ({odometer})\n \t Estado del vehículo: ({condition})\n \t Identificación del vehículo: ({vin})')

Completitud de variables: 
 	 Odómetro: (8205)
 	 Estado del vehículo: (7)
 	 Identificación del vehículo: (7368)


In [293]:
dfVehicles['condition'].unique().tolist()

['excellent', 'like new', 'good', 'fair', 'salvage', 'new', nan]

In [294]:
dfVehicles['odometer'].value_counts()

120000.0    53
150000.0    45
170000.0    44
0.0         44
125000.0    43
            ..
137524.0     1
22932.0      1
62372.0      1
137334.0     1
46.0         1
Name: odometer, Length: 8204, dtype: int64

## Discriminate values

Evalúe en qué medida el valor discrimina su variable dependiente y determine si lo utilizaría en su modelo.

In [295]:
print(dfVehicles['odometer'].min())
print(dfVehicles['odometer'].max())

# # Alternative forms
# dfVehicles.nlargest(2, 'odometer', keep='all')
# dfVehicles.nsmallest(3, 'odometer', keep='all')

0.0
4710000.0


Si la respuesta anterior es afirmativa, entonces cómo propondría incluirlo en su modelo. Es decir, qué transformaciones, técnicas o herramientas utilizarías para incluirlo.

# Data analysis

In [296]:
# Create new categorial value
conditions = [
    (dfVehicles['odometer']>=0) & (dfVehicles['odometer']<=999999),
    (dfVehicles['odometer']>=1000000) & (dfVehicles['odometer']<=1999999),
    (dfVehicles['odometer']>=2000000) & (dfVehicles['odometer']<=2999999),
    (dfVehicles['odometer']>=3000000) & (dfVehicles['odometer']<=3999999),
    (dfVehicles['odometer']>=4000000) & (dfVehicles['odometer']<=4999999),
    (dfVehicles['odometer']>=5000000) & (dfVehicles['odometer']<=5999999),
    (dfVehicles['odometer']>=6000000) & (dfVehicles['odometer']<=6999999),
    (dfVehicles['odometer']>=7000000) & (dfVehicles['odometer']<=7999999),
    (dfVehicles['odometer']>=8000000) & (dfVehicles['odometer']<=8999999),
    (dfVehicles['odometer']>=9000000)
]

options = [1,2,3,4,5,6,7,8,9,10]

dfVehicles['cat_odometer'] = np.select(conditions, options)

In [297]:
dfVehicles['cat_odometer'].value_counts()

1    12696
2       13
0        9
3        4
5        1
4        1
Name: cat_odometer, dtype: int64

# Aquí

In [298]:
# # Transform 'value_counts' object in a DataFrame
# dfOdometer = dfVehicles['odometer'].value_counts()
# dfOdometer = dfOdometer.rename_axis('Valor en odómetro')
# dfOdometer = dfOdometer.reset_index(name='Repeticiones')

# dfOdometer.head()

# Multiple regression

## Preprocessing


In [299]:
def label_encoder_fun(var_encoder, encoder_name):
    encoder_name = preprocessing.LabelEncoder()
    var_coded = var_encoder + '_le'
    dfVehicles[var_coded] = encoder_name.fit_transform(dfVehicles[var_encoder])

In [300]:
# Label encoder in multiples variables
labels_to_encoders_name = {
    'condition':'le_cond',
    'transmission':'le_trans'}

for var_encoder, encoder_name in labels_to_encoders_name.items():
    label_encoder_fun(var_encoder, encoder_name)

# le_trans.classes_

In [301]:
# Scaler data
scaler = preprocessing.StandardScaler()
dfVehicles['odometer'] = scaler.fit_transform(dfVehicles['odometer'])

ValueError: Expected 2D array, got 1D array instead:
array=[100000.  61000. 122000. ...     nan     nan     nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Model

In [ ]:
reg = smf.ols('price ~ odometer + condition_le + transmission_le', data=dfVehicles)
res = reg.fit()

In [ ]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.4896
Date:                Mon, 06 Jun 2022   Prob (F-statistic):              0.613
Time:                        03:30:05   Log-Likelihood:            -2.2286e+05
No. Observations:               12715   AIC:                         4.457e+05
Df Residuals:                   12712   BIC:                         4.457e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2.056e+05   1.57e+05      1.313   

In [ ]:
print(res.rsquared)
print(res.params)

7.70162053131962e-05
Intercept       205579.016794
odometer            -0.153473
condition_le    -76399.730660
dtype: float64


# Final

In [ ]:
dfVehicles.head()

,index,price,manufacturer,condition,cylinders,fuel,odometer,transmission,vin,drive,size,type,state,postal_code,State,Number of Crashes,Poverty,cat_odometer,condition_le,transmission_le
0,0.0,3400.0,volkswagen,excellent,4 cylinders,gas,100000.0,automatic,WVWSK61J62W209823,NaN,NaN,wagon,CA,CA,California,116,12.8,1,0,1
1,4.0,28900.0,cadillac,excellent,8 cylinders,gas,61000.0,automatic,NaN,rwd,mid-size,sedan,CA,CA,California,116,12.8,1,0,1
2,7.0,8000.0,ford,like new,8 cylinders,other,122000.0,automatic,NaN,rwd,NaN,van,CA,CA,California,116,12.8,1,3,1
3,25.0,6905.0,NaN,excellent,8 cylinders,gas,183991.0,automatic,1GKFK16Z95J236202,4wd,NaN,SUV,CA,CA,California,116,12.8,1,0,1
4,33.0,38985.0,bmw,excellent,NaN,gas,18966.0,automatic,WBAJE5C54JWA92562,rwd,NaN,sedan,CA,CA,California,116,12.8,1,0,1
